<a href="https://colab.research.google.com/github/GB1111/Coursera_Capstone/blob/master/Pune_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [2]:
path = 'https://raw.githubusercontent.com/GB1111/Coursera_Capstone/master/pune_data.csv'
pune_data = pd.read_csv(path)
pune_data.head()

,Postal Code,Place Name,County,Community,Latitude,Longitude,Accuracy
0,410301,Khandala,Pune,Maval,18.7589,73.3694,4
1,410302,R P T S Khandala,Pune,Maval,18.7589,73.3694,1
2,410401,Lonavala,Pune,Maval,18.7528,73.4057,4
3,410401,Kusgaon BK,Pune,Maval,18.7528,73.4057,3
4,410401,Lonavala Bazar,Pune,Maval,18.7528,73.4057,3


In [3]:
pune_data = pune_data[['Postal Code', 'Community', 'Place Name', 'Latitude', 'Longitude']]
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [4]:
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [5]:
print('The dataframe has {} communities and {} places.'.format(
        len(pune_data['Community'].unique()),
        pune_data.shape[0]
    )
)

The dataframe has 20 communities and 791 places.


In [6]:
pune_data.groupby('Community').count()

,Postal Code,Place Name,Latitude,Longitude
Community,,,,
Ambegaon,43,43,43,43
Baramati,56,56,56,56
Bhor,69,69,69,69
Daund,52,52,52,52
Haveli,76,76,76,76
Indapur,50,50,50,50
Junnar,68,68,68,68
Khed,68,68,68,68
Maval,38,38,38,38


In [7]:
CLIENT_ID = 'NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO' # your Foursquare ID
CLIENT_SECRET = 'QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO
CLIENT_SECRET:QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS


In [8]:
pune_data.loc[0, 'Place Name']

'Khandala'

In [9]:
place_latitude = pune_data.loc[0, 'Latitude'] # neighborhood latitude value
place_longitude = pune_data.loc[0, 'Longitude'] # neighborhood longitude value

place_name = pune_data.loc[0, 'Place Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(place_name, 
                                                               place_latitude, 
                                                               place_longitude))

Latitude and longitude values of Khandala are 18.7589, 73.3694.


In [10]:
LIMIT = 100
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    place_latitude, 
    place_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO&client_secret=QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS&v=20180605&ll=18.7589,73.3694&radius=500&limit=100'

In [0]:
import requests

In [0]:

results = requests.get(url).json()

In [0]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,High Point @ Dukes Retreat,Hotel Bar,18.761639,73.370665
1,Dukes Retreat Khandala,Hotel,18.761514,73.370721
2,Shooting point,Outdoors & Recreation,18.758637,73.373100
3,Velvett Country,Resort,18.755733,73.367764


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
pune_venues = getNearbyVenues(names=pune_data['Place Name'],
                                   latitudes=pune_data['Latitude'],
                                   longitudes=pune_data['Longitude']
                                 )
print(pune_venues.shape)
pune_venues.head()


Khandala
R P T S Khandala
Lonavala
Kusgaon BK
Lonavala Bazar
Ambavane
Ins Shivaji Lonavale
Kurwande
Kaivalyadham
Karanjgaon
Kamshet
Karunj
Malawali
Karla
Takave KH
Vehergaon
Nane
Pawananagar
Shivali
Bhose
Chakan
Mahalunge Ingale
Vasuli
Waki BK
Ambethan
Vadgaon Ghenanad
Khalumbre
Shive
Rase
Pimpalgaon Tarfe Chakan
Kuruli
Koyali
Nighoje
Kalus
Gadad
Bahul
Rajur
Parunde
Junnar
Tambe
Devale
Inglun
Khanapur
Vadaj
Anjanavale
Kusur
Khangaon
Amboli
Aptale
Yenere
Khamgaon
Khadkumbe(Exptl)
Ballalwadi
Manikdoh
Chavand
Nimgiri
Godre
Nirgude
Utchil
Manchar
Landewadi
Chandoli BK
Vadgaon Kashimbeg
Pimpalgaon Mahalunge
Narodi
Ozar
Khodad
Yedgaon
Pimparkhed
Pargaon Tarfe Ala
Ranjani
Valati
Narayangaon
Nimgaon Sava
Khamundi
Pimpri Pendhar
Jambhut
Hivare BK
Amboli
Vaphgaon
Pimpri BK
Rajgurunagar
Palu
Donda
Nimgaon Khed
Chinchoshi
Pait
Yeniye BK
Kurkundi
Varude
Kude BK
Sandbhorwadi(Exptl)
Kiwale
Kanhersar
Retwadi
Shiroli
Gulani
Ghotawadi
Dawadi
Talegaon Dabhade
Dhamane
Urse
Chandkhed
Adhale BK
Bebedhol
Ord

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar
1,Khandala,18.7589,73.3694,Dukes Retreat Khandala,18.761514,73.370721,Hotel
2,Khandala,18.7589,73.3694,Shooting point,18.758637,73.373100,Outdoors & Recreation
3,Khandala,18.7589,73.3694,Velvett Country,18.755733,73.367764,Resort
4,R P T S Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar


In [19]:
pune_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar
1,Khandala,18.7589,73.3694,Dukes Retreat Khandala,18.761514,73.370721,Hotel
2,Khandala,18.7589,73.3694,Shooting point,18.758637,73.373100,Outdoors & Recreation
3,Khandala,18.7589,73.3694,Velvett Country,18.755733,73.367764,Resort
4,R P T S Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar


In [20]:
pune_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
9 DRD,32,32,32,32,32,32
A.R. Shala,15,15,15,15,15,15
Adhale BK,4,4,4,4,4,4
Airport (Pune),4,4,4,4,4,4
Akurdi,4,4,4,4,4,4
...,...,...,...,...,...,...
Waki,2,2,2,2,2,2
Warje,5,5,5,5,5,5
Yavat,1,1,1,1,1,1


In [21]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 85 uniques categories.


**Analyze Each Neighborhood**

In [22]:
# one hot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
pune_onehot['Neighbourhood'] = pune_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[fixed_columns]

pune_onehot.head()

,Neighbourhood,ATM,Antique Shop,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bed & Breakfast,Bistro,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gym,Gym / Fitness Center,Hotel,Hotel Bar,...,Juice Bar,Lake,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Nature Preserve,Nightclub,North Indian Restaurant,Outdoors & Recreation,Outlet Mall,Park,Pizza Place,Platform,Resort,Rest Area,Restaurant,River,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Stables,Stadium,Tea Room,Tennis Court,Theater,Track,Trail,Train Station,Vegetarian / Vegan Restaurant
0,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,R P T S Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
pune_onehot.shape

(877, 86)

In [24]:
pune_grouped = pune_onehot.groupby('Neighbourhood').mean().reset_index()
pune_grouped

,Neighbourhood,ATM,Antique Shop,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bed & Breakfast,Bistro,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gym,Gym / Fitness Center,Hotel,Hotel Bar,...,Juice Bar,Lake,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Nature Preserve,Nightclub,North Indian Restaurant,Outdoors & Recreation,Outlet Mall,Park,Pizza Place,Platform,Resort,Rest Area,Restaurant,River,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Stables,Stadium,Tea Room,Tennis Court,Theater,Track,Trail,Train Station,Vegetarian / Vegan Restaurant
0,9 DRD,0.0,0.0,0.0,0.0,0.0,0.03125,0.000000,0.000000,0.0,0.03125,0.0,0.00,0.0,0.062500,0.03125,0.062500,0.0,0.000000,0.09375,0.0,0.0,0.000000,0.0,0.0625,0.0,0.0,0.0,0.0,0.03125,0.0,0.03125,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.062500,0.0,...,0.03125,0.0,0.031250,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.03125,0.03125,0.0,0.0,0.0,0.062500,0.0,0.0,0.0,0.0,0.00,0.03125,0.03125,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,A.R. Shala,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.066667,0.0,0.00000,0.0,0.00,0.0,0.066667,0.00000,0.066667,0.0,0.066667,0.00000,0.0,0.0,0.066667,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.066667,0.0,0.066667,0.066667,0.066667,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.20,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0
2,Adhale BK,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.25,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.0
3,Airport (Pune),0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.00000,0.00000,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.00,0.25000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Akurdi,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.25000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.25000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Waki,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...

In [25]:

pune_grouped.shape

(176, 86)

In [26]:
num_top_venues = 5

for hood in pune_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = pune_grouped[pune_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----9 DRD----
                venue  freq
0   Indian Restaurant  0.16
1         Coffee Shop  0.09
2  Chinese Restaurant  0.06
3          Restaurant  0.06
4        Dessert Shop  0.06


----A.R. Shala----
                  venue  freq
0           Snack Place  0.20
1                  Café  0.07
2        Clothing Store  0.07
3    Chinese Restaurant  0.07
4  Gym / Fitness Center  0.07


----Adhale BK----
               venue  freq
0      Train Station  0.50
1        Bus Station  0.25
2  Indian Restaurant  0.25
3    Motorcycle Shop  0.00
4               Park  0.00


----Airport (Pune)----
          venue  freq
0    Restaurant  0.50
1   Outlet Mall  0.25
2  Soccer Field  0.25
3           ATM  0.00
4      Platform  0.00


----Akurdi----
                            venue  freq
0               Indian Restaurant  0.25
1                     Coffee Shop  0.25
2            Fast Food Restaurant  0.25
3                     Snack Place  0.25
4  Multicuisine Indian Restaurant  0.00


----Ala (Pune)----


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = pune_grouped['Neighbourhood']

for ind in np.arange(pune_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pune_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9 DRD,Indian Restaurant,Coffee Shop,Hotel,Ice Cream Shop,Chinese Restaurant,Restaurant,Café,Dessert Shop,Juice Bar,Fast Food Restaurant
1,A.R. Shala,Snack Place,Garden,Clothing Store,Chinese Restaurant,Restaurant,Café,Bistro,Deli / Bodega,Indian Restaurant,Hotel
2,Adhale BK,Train Station,Bus Station,Indian Restaurant,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest
3,Airport (Pune),Restaurant,Outlet Mall,Soccer Field,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market
4,Akurdi,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Snack Place,Vegetarian / Vegan Restaurant,Forest,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm


In [29]:
neighbourhoods_venues_sorted.shape

(176, 11)

Cluster Neighborhoods


In [30]:
# set number of clusters
kclusters = 5

pune_grouped_clustering = pune_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 2, 0, 4, 0, 2, 2], dtype=int32)

In [0]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
pune_merged = pune_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pune_merged = pune_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Place Name')

pune_merged.head() # check the last columns!

,Postal Code,Community,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,410301,Maval,Khandala,18.7589,73.3694,2.0,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
1,410302,Maval,R P T S Khandala,18.7589,73.3694,2.0,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
2,410401,Maval,Lonavala,18.7528,73.4057,2.0,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
3,410401,Maval,Kusgaon BK,18.7528,73.4057,2.0,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
4,410401,Maval,Lonavala Bazar,18.7528,73.4057,2.0,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant


In [34]:
pune_merged.shape

(791, 16)

In [35]:
pune_merged.dropna(inplace=True)
pune_merged['Cluster Labels'] = pune_merged['Cluster Labels'].astype(int)
pune_merged.head()
#pune_merged.shape

,Postal Code,Community,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,410301,Maval,Khandala,18.7589,73.3694,2,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
1,410302,Maval,R P T S Khandala,18.7589,73.3694,2,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
2,410401,Maval,Lonavala,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
3,410401,Maval,Kusgaon BK,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
4,410401,Maval,Lonavala Bazar,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant


In [37]:
#!pip install geocoder

     |████████████████████████████████| 102kB 3.1MB/s 


In [0]:
import geocoder

In [41]:
address = 'Pune, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune are 18.521428, 73.8544541.


In [0]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Place Name'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
Clusters_pune = pune_merged.groupby("Cluster Labels")
Clusters_pune.head()

,Postal Code,Community,Place Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,410301,Maval,Khandala,18.7589,73.3694,2,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
1,410302,Maval,R P T S Khandala,18.7589,73.3694,2,Hotel Bar,Hotel,Resort,Outdoors & Recreation,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store
2,410401,Maval,Lonavala,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
3,410401,Maval,Kusgaon BK,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
4,410401,Maval,Lonavala Bazar,18.7528,73.4057,2,Hotel,Indian Restaurant,Dessert Shop,Fast Food Restaurant,Café,Hotel Pool,Ice Cream Shop,Train Station,Lake,Multicuisine Indian Restaurant
20,410501,Khed,Chakan,18.7606,73.8635,1,North Indian Restaurant,Restaurant,Men's Store,Vegetarian / Vegan Restaurant,Forest,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market
59,410503,Ambegaon,Manchar,19.0044,73.9435,3,Indie Movie Theater,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest
60,410503,Ambegaon,Landewadi,19.0044,73.9435,3,Indie Movie Theater,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest
61,410503,Ambegaon,Chandoli BK,19.0044,73.9435,3,Indie Movie Theater,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest
62,410503,Ambegaon,Vadgaon Kashimbeg,19.0044,73.9435,3,Indie Movie Theater,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest
